### Ensure to run this file in the uppermost folder

In [12]:
cd /home/ubuntu/pFedDef_v1/

/home/ubuntu/pFedDef_v1


In [8]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

### Load Regular
Input may be changed include:
1. setting (FedEM, local)
2. args_.experiment (which dataset is used)
3. args_.save_path (path to the folder that contain the trained model)

In [9]:
# Generating Empty Aggregator to be loaded 

setting = 'FedEM'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1

# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/dummy/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 269.68it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:37<00:00,  2.13it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.291 | Train Acc: 12.564% |Test Loss: 2.291 | Test Acc: 13.015% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [10]:
# Compiling Dataset from Clients
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].test_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
try:
    data_y = torch.stack(data_y)        
except:
    data_y = torch.FloatTensor(data_y) 
    
dataloader = Custom_Dataloader(data_x, data_y)

### Import model weights

In [5]:
# Import Model Weights
num_models = 40

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':

    aggregator.load_state(args_.save_path)
    
    model_weights = []
    weights = np.load(args_.save_path+"/train_client_weights.npy")
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

    weights = np.load(args_.save_path+"/train_client_weights.npy")
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

    weights = np.load(args_.save_path+"/train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

### Trials setup

In [16]:
# Set trials 
num_trials = 50
batch_size = 500
adv_idx = 0
adv_idxs = [0,1]
model_idx_max = 8



### Setup attack

In [26]:
dists_measure_legit = np.zeros([num_trials, model_idx_max])
dists_measure_adv = np.zeros([num_trials, model_idx_max])
dists_measure_adv_ensemble = np.zeros([num_trials, model_idx_max])


# attack parameters
eps = 4.5
iteration = 10
target = -1
eps_norm = 2
step_size = 0.01

for i in range(num_trials):
    print("num_trial:", i)
    t1 = Boundary_Transferer(models_list=models_test[:model_idx_max], dataloader=dataloader)
    t1.base_nn_idx = 0
    t1.victim_idx = range(model_idx_max)

    t1.atk_params = IFSGM_Params()
    t1.atk_params.set_params(batch_size=1, eps=0.1, alpha=0.01, iteration = 30,
                       target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x))
    
    t1.set_adv_NN(t1.base_nn_idx)

    base_ep_legit, victim_eps_legit = t1.legitimate_direction(batch_size=batch_size, ep_granularity = 0.1, 
                                                              rep_padding = 1000, new_point = True,print_res = False)
    
    base_ep_adv, victim_eps_adv = t1.adversarial_direction(ep_granularity = 0.1, 
                                                              rep_padding = 1000, new_point = False,print_res = False)

    t1.set_ensemble_adv_NN(adv_idxs)
    base_ep_Eadv, victim_eps_Eadv = t1.ensemble_adversarial_direction(ep_granularity = 0.1, 
                                                              rep_padding = 1000, new_point = False,print_res = False)
    
    idx = 0
    for key, value in victim_eps_legit.items():
        dists_measure_legit[i,idx] = np.abs(base_ep_legit-value)
        idx+=1
        
    idx = 0
    for key, value in victim_eps_adv.items():
        dists_measure_adv[i,idx] = np.abs(base_ep_adv - value)
        idx+=1
        
    idx = 0
    for key, value in victim_eps_Eadv.items():
        dists_measure_adv_ensemble[i,idx] = np.abs(base_ep_Eadv - value)
        idx+=1
    
    

num_trial: 0
num_trial: 1
num_trial: 2
num_trial: 3
num_trial: 4
num_trial: 5
num_trial: 6
num_trial: 7
num_trial: 8
num_trial: 9
num_trial: 10
num_trial: 11
num_trial: 12
num_trial: 13
num_trial: 14
num_trial: 15
num_trial: 16
num_trial: 17
num_trial: 18
num_trial: 19
num_trial: 20
num_trial: 21
num_trial: 22
num_trial: 23
num_trial: 24
num_trial: 25
num_trial: 26
num_trial: 27
num_trial: 28
num_trial: 29
num_trial: 30
num_trial: 31
num_trial: 32
num_trial: 33
num_trial: 34
num_trial: 35
num_trial: 36
num_trial: 37
num_trial: 38
num_trial: 39
num_trial: 40
num_trial: 41
num_trial: 42
num_trial: 43
num_trial: 44
num_trial: 45
num_trial: 46
num_trial: 47
num_trial: 48
num_trial: 49


In [27]:
np.average(dists_measure_legit)

array([0.00, 7.17, 0.36, 10.04])

In [28]:
np.average(dists_measure_adv)

array([0.19, 2.62, 0.24, 3.95])